### Word Frequency

In [1]:
import string
import pandas as pd
import numpy as np
import nltk
import re
from itertools import combinations, product

nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
translator = str.maketrans('', '', string.punctuation)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/abhivik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
postsDF = pd.read_csv('./scrappy/scrappy_do.csv')

def wordCount(post):
    d = {}
    l = str(post).split(" ")
    filtered = [w for w in l if w not in stop_words]
    for w in filtered:
        w = w.translate(translator) # remove punctuation
        w = w.lower()
        if (d.get(w)):
            d[w] = d[w] + 1
        else:
            d[w] = 1
    return d

oneString = ' '.join(list(map(lambda x: str(x), list(postsDF['message']))))
wordCounts = wordCount(oneString)

In [3]:
# ordered words by frequency count to determine a list of important words
wordCountDF = pd.DataFrame.from_dict(wordCounts, orient="index")
print(list(wordCountDF.sort_values(by=0, ascending=False).reset_index()['index'])[:200])

['i', '', 'car', 'the', 'like', 'cars', 'bmw', 'would', 'one', 'it', 'get', 'new', 'think', 'tl', 'even', 'much', 'im', 'better', 'good', 'drive', 'well', '3', 'really', 'know', 'and', 'people', 'time', 'also', 'price', 'audi', 'want', 'performance', 'but', 'if', 'years', 'still', 'acura', 'driving', 'you', 'see', 'could', 'that', 'way', 'is', 'luxury', 'make', 'back', 'say', 'go', 'lexus', 'engine', 'series', 'thats', 'buy', 'going', 'sedan', 'g35', 'not', 'many', 'so', 'great', 'sport', 'less', 'ive', 'my', 'manual', 'miles', 'right', 'best', 'said', 'year', 'me', 'got', 'in', 'awd', 'take', 'lot', 'interior', 'its', 'look', 'infiniti', 'never', 'cts', 'may', 'sure', 'us', 'since', 'cant', 'nice', 'they', 'dealer', 'test', 'thing', 'first', 'need', 'something', 'probably', 'point', 'two', 'power', 'this', 'little', 'last', 'every', 'looking', 'rwd', 'around', 'id', 'used', 'now', 'feel', 'tires', '2', 'money', 'a', 'least', 'class', 'actually', '5', 'there', 'cost', 'old', 'seems', '

In [4]:
modelsDF = pd.read_csv('./models.csv', names=['make', 'model'])
model_to_make = modelsDF.set_index('model').to_dict(orient='dict')['make']

# importantWords = ['performance', 'luxury sport', 'quality','experience' ,'power','handling', 'premium',  'reliable', 'expensive', 'worth', 'speed', 'family']
importantWords = ['performance', 'luxury', 'power', 'handling', 'speed']

brands_unclean = list(set(model_to_make.values()))
brandlist = [e for e in brands_unclean if e not in ('car', 'reliable', 'seat', 'speed', 'sedan', 'problem')]
models = list(model_to_make.keys())
keepWords = list(set(importantWords + brands_unclean + models))
print(keepWords[:100])

['expedition', 'escalade', 'suzuki', '335i', 'dodge', '3-series', 'zest', 'pathfinder', 'gxp', 'tribeca', 'rsx', 'altimas', '4-series', 'geo metro', 'tundra', 'fortuner', 'sonatas', 'sai', 'audi', 'succeed', 'sofia', 'q70', 'vws', 'toyoace', 'forte', 'biante', 'rav4', 'c-max', 'streamliner', 'stream', 'volt', 'cervo', 'escudo', 'fronte', 'subaru', 'cx-9', 'tsx', '6-series', 'dart', 'g37', 'fords', 'vibe', 'brz', 'sorento', 'vanguard', 'srx', 'noah', '5er', 'la crosse', 'grandeur', 'aztek', 'probox', 's-10', 'altima', 'genesis', 'cappuccino', 'lacetti', 'splash', '6mt', 'mazda6s', 'zio', 'vios', 'm5', 'venture', 'avensis', 'lantra', 'pacifica', 'qx50', 'impala', 'caliber', 'xterra', 'mazda5', 'xts', 'echo', 'porte', 'tavera', 'cr-v', 'mark x', 'santro', 'townace', 'platz', 'm3', 'touraeg', 'mkx', 'town and country', 'c70', 'allion', 'granvia', 'click', 'xc90', 'cruze', 'carri', 'charger', 'sienta', 'accords', 'scrum', 'beetle', 'infiniti', 'xcent', '535i']


In [5]:
def stopUnique(post):
    # lowercase
    post = str(post).lower()
    
    # remove punctuation
    post = post.translate(translator)
    modelKeys = model_to_make.keys()
    wordList = post.split(' ')
    
    # remove stop words
    filtered = [w for w in wordList if w not in stop_words]
    
    # only keep important words
    finalWords = list(filter(lambda x: x in keepWords, filtered))
    
    # make / model replacement
    for idx, word in enumerate(finalWords):
        if word in modelKeys:
            finalWords[idx] = model_to_make.get(word)
            
    unique = list(set(finalWords))
    return unique if len(unique) else []
    

postsDF['words'] = postsDF['message'].map(lambda x: stopUnique(x))
len(postsDF)

21664

In [ ]:
dataset = list(postsDF['words'])
dataset[:100]

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori


te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.015, use_colnames=True)


frequent_itemsets


In [ ]:
frequent_itemsets = frequent_itemsets[0:84]
frequent_itemsets

In [ ]:
from mlxtend.frequent_patterns import association_rules
result_lift = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)


In [ ]:
result_lift.sort_values(by=['lift'],ascending=False)

In [6]:
def do_lift(x, y=None):
    l1, l2 = sorted(x), sorted(y or [])
    bools = pd.DataFrame()
    for e in l1 + l2:
        bools[e] = postsDF['words'].map(lambda words: e in words)

    def do(df, a, b):
        count_a_int_b = np.logical_and(df[a], df[b]).values.sum()
        count_a_mult_count_b = df[a].values.sum() * df[b].values.sum()
        return len(df.index) * count_a_int_b / count_a_mult_count_b

    result_df = pd.DataFrame(index=l1, columns=(l2 or l1))
    for a, b in (combinations(l1, 2) if y==None else product(l1, l2)):
        result_df.at[a, b] = do(bools, a, b)

    return result_df

In [7]:
do_lift(brandlist)

,acura,audi,bmw,buick,cadillac,chevrolet,chrysler,dodge,ford,honda,...,mercury,mitsubishi,nissan,pontiac,saturn,subaru,suzuki,toyota,volkswagen,volvo
acura,NaN,2.25881,1.35088,2.51953,2.308,1.34404,1.7445,0.985305,1.40097,3.25366,...,1.85505,1.84805,1.99418,1.69344,1.57134,1.6468,3.47482,1.72194,1.72271,2.50784
audi,NaN,NaN,1.66233,1.80034,3.05186,1.58915,1.86342,1.12969,1.53773,1.71347,...,2.15348,1.62527,1.23376,1.42685,0.760051,1.53754,2.52115,1.57082,3.34349,3.1757
bmw,NaN,NaN,NaN,1.30356,1.83762,1.38111,1.30014,1.17196,1.4457,1.18766,...,1.20657,1.43423,1.25842,1.24358,1.49047,1.19879,1.67743,1.4399,1.28244,1.48912
buick,NaN,NaN,NaN,NaN,5.12828,6.13351,6.25511,3.99684,3.58307,2.40983,...,31.9271,3.94297,2.26755,8.97449,14.3416,1.49805,5.09701,3.48591,1.85309,4.40881
cadillac,NaN,NaN,NaN,NaN,NaN,4.04018,3.52613,2.72352,2.51339,1.58053,...,9.69121,2.5077,1.34119,4.07693,8.79538,1.28621,2.43124,1.97276,1.52675,3.52248
chevrolet,NaN,NaN,NaN,NaN,NaN,NaN,5.56129,5.95635,4.80969,1.84124,...,13.2467,3.42771,2.77944,7.24446,10.6864,2.14877,5.53868,2.54244,2.41639,2.01215
chrysler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.5967,5.21718,2.62884,...,14.7374,8.34193,2.99354,5.42482,10.4029,2.66224,8.62678,3.67498,2.45921,2.79824
dodge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.56695,1.72774,...,25.8962,10.0513,2.8902,7.08116,10.4455,2.86409,6.4966,2.25921,2.1472,1.40486
ford,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.17907,...,15.0217,3.09194,2.26179,3.82035,3.85583,2.8898,3.9969,2.72427,2.85341,3.42266
honda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.90342,3.53518,3.15628,2.15526,2.75536,1.8132,3.99863,3.4366,2.37887,1.77878


In [8]:
do_lift(brandlist, importantWords)

,handling,luxury,performance,power,speed
acura,1.6829,2.03727,1.62576,1.85756,1.83004
audi,1.85199,1.89727,1.73898,1.93522,1.84652
bmw,1.79176,1.46202,1.62448,1.62643,1.3359
buick,2.57739,2.42256,1.72338,1.58851,1.00063
cadillac,1.82748,2.81791,1.72629,1.31898,1.39517
chevrolet,2.36169,1.49636,1.61803,1.49601,2.00739
chrysler,1.50322,2.53566,1.40009,1.71771,1.66101
dodge,1.68695,1.19177,1.44976,1.72475,1.96214
ford,1.73705,2.35961,1.99469,1.52247,1.84697
honda,1.67457,1.97196,1.51231,1.80006,1.95387
